# Sumarização com similaridade do cosseno

## Preparação do texto de exemplo

In [1]:
import re
import nltk
import spacy
import string

import numpy    as np
import networkx as nx

from goose3               import Goose
from nltk.cluster.util    import cosine_distance
from IPython.core.display import HTML

In [2]:
# nltk.download()

In [5]:
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /home/junior/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /home/junior/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /home/junior/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [6]:
stopwords = nltk.corpus.stopwords.words('portuguese')
print(stopwords)

['a', 'à', 'ao', 'aos', 'aquela', 'aquelas', 'aquele', 'aqueles', 'aquilo', 'as', 'às', 'até', 'com', 'como', 'da', 'das', 'de', 'dela', 'delas', 'dele', 'deles', 'depois', 'do', 'dos', 'e', 'é', 'ela', 'elas', 'ele', 'eles', 'em', 'entre', 'era', 'eram', 'éramos', 'essa', 'essas', 'esse', 'esses', 'esta', 'está', 'estamos', 'estão', 'estar', 'estas', 'estava', 'estavam', 'estávamos', 'este', 'esteja', 'estejam', 'estejamos', 'estes', 'esteve', 'estive', 'estivemos', 'estiver', 'estivera', 'estiveram', 'estivéramos', 'estiverem', 'estivermos', 'estivesse', 'estivessem', 'estivéssemos', 'estou', 'eu', 'foi', 'fomos', 'for', 'fora', 'foram', 'fôramos', 'forem', 'formos', 'fosse', 'fossem', 'fôssemos', 'fui', 'há', 'haja', 'hajam', 'hajamos', 'hão', 'havemos', 'haver', 'hei', 'houve', 'houvemos', 'houver', 'houvera', 'houverá', 'houveram', 'houvéramos', 'houverão', 'houverei', 'houverem', 'houveremos', 'houveria', 'houveriam', 'houveríamos', 'houvermos', 'houvesse', 'houvessem', 'houvésse

In [7]:
def preprocessamento(texto):
    texto_formatado = texto.lower()
    tokens = []
    
    for token in nltk.word_tokenize(texto_formatado):
        tokens.append(token)

    tokens = [palavra for palavra in tokens if palavra not in stopwords and palavra not in string.punctuation]
    texto_formatado = ' '.join([str(elemento) for elemento in tokens if not elemento.isdigit()])

    return texto_formatado

In [8]:
texto_original = """A inteligência artificial é a inteligência similar à humana máquinas.
                    Definem como o estudo de agente artificial com inteligência.
                    Ciência e engenharia de produzir máquinas com inteligência.
                    Resolver problemas e possuir inteligência.
                    Relacionada ao comportamento inteligente.
                    Construção de máquinas para raciocinar.
                    Aprender com os erros e acertos.
                    Inteligência artificial é raciocinar nas situações do cotidiano."""

texto_original = re.sub(r'\s+', ' ', texto_original)
texto_original

'A inteligência artificial é a inteligência similar à humana máquinas. Definem como o estudo de agente artificial com inteligência. Ciência e engenharia de produzir máquinas com inteligência. Resolver problemas e possuir inteligência. Relacionada ao comportamento inteligente. Construção de máquinas para raciocinar. Aprender com os erros e acertos. Inteligência artificial é raciocinar nas situações do cotidiano.'

## Função para calcular a similaridade entre sentenças

- Link: https://en.wikipedia.org/wiki/Cosine_similarity
- Cálculos passo a passo: https://janav.wordpress.com/2013/10/27/tf-idf-and-cosine-similarity/

In [9]:
sentencas_originais = [sentenca for sentenca in nltk.sent_tokenize(texto_original)]
sentencas_formatadas = [preprocessamento(sentenca_original) for sentenca_original in sentencas_originais]

In [10]:
sentencas_originais

['A inteligência artificial é a inteligência similar à humana máquinas.',
 'Definem como o estudo de agente artificial com inteligência.',
 'Ciência e engenharia de produzir máquinas com inteligência.',
 'Resolver problemas e possuir inteligência.',
 'Relacionada ao comportamento inteligente.',
 'Construção de máquinas para raciocinar.',
 'Aprender com os erros e acertos.',
 'Inteligência artificial é raciocinar nas situações do cotidiano.']

In [11]:
sentencas_formatadas

['inteligência artificial inteligência similar humana máquinas',
 'definem estudo agente artificial inteligência',
 'ciência engenharia produzir máquinas inteligência',
 'resolver problemas possuir inteligência',
 'relacionada comportamento inteligente',
 'construção máquinas raciocinar',
 'aprender erros acertos',
 'inteligência artificial raciocinar situações cotidiano']

In [12]:
def calcula_similaridade_sentencas(sentenca1, sentenca2):
    
    palavras1 = [palavra for palavra in nltk.word_tokenize(sentenca1)]
    palavras2 = [palavra for palavra in nltk.word_tokenize(sentenca2)]
    #print(palavras1)
    #print(palavras2)

    todas_palavras = list(set(palavras1 + palavras2))
    #print(todas_palavras)

    vetor1 = [0] * len(todas_palavras)
    vetor2 = [0] * len(todas_palavras)
    #print(vetor1)
    #print(vetor2)

    for palavra in palavras1:
        vetor1[todas_palavras.index(palavra)] += 1
    
    for palavra in palavras2:
        vetor2[todas_palavras.index(palavra)] += 1

    #print(vetor1)
    #print(vetor2) K-nn
    
    return 1 - cosine_distance(vetor1, vetor2)

In [13]:
calcula_similaridade_sentencas(sentencas_formatadas[0], sentencas_formatadas[0])

np.float64(0.9999999999999998)

## Função para gerar a matriz de similaridade

In [14]:
def calcula_matriz_similaridade(sentencas):
  
    matriz_similaridade = np.zeros((len(sentencas), len(sentencas)))
    
    #print(matriz_similaridade)
  
    for i in range(len(sentencas)):
        for j in range(len(sentencas)):
            if i == j:
                continue
    
            matriz_similaridade[i][j] = calcula_similaridade_sentencas(sentencas[i], sentencas[j])

    return matriz_similaridade

In [15]:
calcula_matriz_similaridade(sentencas_formatadas)

array([[0.        , 0.47434165, 0.47434165, 0.35355339, 0.        ,
        0.20412415, 0.        , 0.47434165],
       [0.47434165, 0.        , 0.2       , 0.2236068 , 0.        ,
        0.        , 0.        , 0.4       ],
       [0.47434165, 0.2       , 0.        , 0.2236068 , 0.        ,
        0.25819889, 0.        , 0.2       ],
       [0.35355339, 0.2236068 , 0.2236068 , 0.        , 0.        ,
        0.        , 0.        , 0.2236068 ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        ],
       [0.20412415, 0.        , 0.25819889, 0.        , 0.        ,
        0.        , 0.        , 0.25819889],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        ],
       [0.47434165, 0.4       , 0.2       , 0.2236068 , 0.        ,
        0.25819889, 0.        , 0.        ]])

## Função para sumarizar

- Algoritmo Page Rank: https://www.youtube.com/watch?v=YfDNI1jp5sM e https://www.youtube.com/watch?v=YplmCue8XJU

In [16]:
def sumarizar(texto, quantidade_sentencas):
    
    sentencas_originais = [sentenca for sentenca in nltk.sent_tokenize(texto)]
    
    sentencas_formatadas = [preprocessamento(sentenca_original) for sentenca_original in sentencas_originais]
    
    matriz_similaridade = calcula_matriz_similaridade(sentencas_formatadas)
    #print(matriz_similaridade)
    
    grafo_similaridade = nx.from_numpy_array(matriz_similaridade)
    #print(grafo_similaridade.nodes)
    #print(grafo_similaridade.edges)
    
    notas = nx.pagerank(grafo_similaridade)
    #print(notas)
    
    notas_ordenadas = sorted(((notas[i], nota) for i, nota in enumerate(sentencas_originais)), reverse=True)
    #print(notas_ordenadas)
    
    melhores_sentencas = []
    
    for i in range(quantidade_sentencas):
        melhores_sentencas.append(notas_ordenadas[i][1])
    
    return sentencas_originais, melhores_sentencas, notas_ordenadas

In [17]:
sentencas_originais, melhores_sentencas, notas_sentencas = sumarizar(texto_original, 3)

In [18]:
sentencas_originais

['A inteligência artificial é a inteligência similar à humana máquinas.',
 'Definem como o estudo de agente artificial com inteligência.',
 'Ciência e engenharia de produzir máquinas com inteligência.',
 'Resolver problemas e possuir inteligência.',
 'Relacionada ao comportamento inteligente.',
 'Construção de máquinas para raciocinar.',
 'Aprender com os erros e acertos.',
 'Inteligência artificial é raciocinar nas situações do cotidiano.']

In [19]:
melhores_sentencas

['A inteligência artificial é a inteligência similar à humana máquinas.',
 'Inteligência artificial é raciocinar nas situações do cotidiano.',
 'Ciência e engenharia de produzir máquinas com inteligência.']

In [20]:
notas_sentencas

[(0.22820924040178003,
  'A inteligência artificial é a inteligência similar à humana máquinas.'),
 (0.1839190802151221,
  'Inteligência artificial é raciocinar nas situações do cotidiano.'),
 (0.1633191450783496,
  'Ciência e engenharia de produzir máquinas com inteligência.'),
 (0.1543717033327776,
  'Definem como o estudo de agente artificial com inteligência.'),
 (0.12639273963873288, 'Resolver problemas e possuir inteligência.'),
 (0.09616903563964783, 'Construção de máquinas para raciocinar.'),
 (0.023809527846794958, 'Relacionada ao comportamento inteligente.'),
 (0.023809527846794958, 'Aprender com os erros e acertos.')]

## Visualização do resumo

In [21]:
def visualiza_resumo(titulo, lista_sentencas, melhores_sentencas):
  
    texto = ''

    display(HTML(f'<h1>Resumo do texto - {titulo}</h1>'))
  
    for i in lista_sentencas:
        
        if i in melhores_sentencas:
            texto += str(i).replace(i, f"<mark>{i}</mark>")
    
        else:
            texto += i
    
    display(HTML(f""" {texto} """))

In [22]:
visualiza_resumo('Teste', sentencas_originais, melhores_sentencas)

## Extração de texto da internet

In [23]:
g = Goose()
url = 'https://iaexpert.academy/2020/11/09/ia-preve-resultado-das-eleicoes-americanas/'
artigo = g.extract(url)

In [24]:
artigo.cleaned_text

'Nas eleições presidenciais americanas de 2016, a maioria das predições apontavam para a vitória de Hillary Clinton. Entretanto, a história nos mostrou o resultado oposto, e Donald Trump foi o presidente nos últimos 4 anos. Desta vez, os estatísticos reexaminaram seus modelos, para aumentar o grau de confiabilidade nos seus resultados. Nesta tentativa de otimização das predições, a inteligência artificial certamente não ficou de fora.\n\nO modelo desenvolvido pelo Dr. Hernan Makse, físico estatístico da Universidade da Cidade de Nova York, baseou suas predições em uma rede neural treinada para processar os sentimentos expressos nas redes sociais. O algoritmo fez a análise de cerca de 1 bilhão de tweets para chegar a uma estimativa dos resultados do pleito. No dia da eleição, 3 de novembro, o modelo estava indicando a vitória de Joe Biden.\n\nO Dr. Makse disse que seu trabalho começou já na eleição de 2016, e foi testado novamente nas eleições na Argentina ano passado. Desta vez, o mode

In [25]:
sentencas_originais, melhores_sentencas, notas_sentencas = sumarizar(artigo.cleaned_text, 5)

In [26]:
sentencas_originais

['Nas eleições presidenciais americanas de 2016, a maioria das predições apontavam para a vitória de Hillary Clinton.',
 'Entretanto, a história nos mostrou o resultado oposto, e Donald Trump foi o presidente nos últimos 4 anos.',
 'Desta vez, os estatísticos reexaminaram seus modelos, para aumentar o grau de confiabilidade nos seus resultados.',
 'Nesta tentativa de otimização das predições, a inteligência artificial certamente não ficou de fora.',
 'O modelo desenvolvido pelo Dr. Hernan Makse, físico estatístico da Universidade da Cidade de Nova York, baseou suas predições em uma rede neural treinada para processar os sentimentos expressos nas redes sociais.',
 'O algoritmo fez a análise de cerca de 1 bilhão de tweets para chegar a uma estimativa dos resultados do pleito.',
 'No dia da eleição, 3 de novembro, o modelo estava indicando a vitória de Joe Biden.',
 'O Dr. Makse disse que seu trabalho começou já na eleição de 2016, e foi testado novamente nas eleições na Argentina ano pas

In [27]:
melhores_sentencas

['Desta vez, o modelo está treinando com cerca de 5 vezes mais dados que nas eleições americanas anteriores.',
 'Quando seu modelo foi usado para predizer os resultados da eleição corrente usando dados brutos, Joe Biden apareceu como vencedor com larga vantagem.',
 'No dia da eleição, 3 de novembro, o modelo estava indicando a vitória de Joe Biden.',
 'Segundo o Dr. Makse, integrar estas duas variáveis em seu modelo é a parte mais importante do trabalho.',
 'O modelo desenvolvido pelo Dr. Hernan Makse, físico estatístico da Universidade da Cidade de Nova York, baseou suas predições em uma rede neural treinada para processar os sentimentos expressos nas redes sociais.']

In [28]:
notas_sentencas

[(0.10322707053208108,
  'Desta vez, o modelo está treinando com cerca de 5 vezes mais dados que nas eleições americanas anteriores.'),
 (0.08467351062155114,
  'Quando seu modelo foi usado para predizer os resultados da eleição corrente usando dados brutos, Joe Biden apareceu como vencedor com larga vantagem.'),
 (0.07824901512415162,
  'No dia da eleição, 3 de novembro, o modelo estava indicando a vitória de Joe Biden.'),
 (0.07627442688432466,
  'Segundo o Dr. Makse, integrar estas duas variáveis em seu modelo é a parte mais importante do trabalho.'),
 (0.07333552035097592,
  'O modelo desenvolvido pelo Dr. Hernan Makse, físico estatístico da Universidade da Cidade de Nova York, baseou suas predições em uma rede neural treinada para processar os sentimentos expressos nas redes sociais.'),
 (0.07240389589770964,
  'Parece que, desta vez, os algoritmos estão de fato contribuindo para que as predições sejam mais precisas.'),
 (0.06666298135659274,
  'O Dr. Makse disse que seu trabalho 

In [29]:
visualiza_resumo('Teste', sentencas_originais, melhores_sentencas)

## Solução para o exercício - lematização

In [30]:
pln = spacy.load("pt_core_news_sm")
pln

In [31]:
def preprocessamento_lematizacao(texto):
  texto = texto.lower()
  texto = re.sub(r" +", ' ', texto)

  documento = pln(texto)
  tokens = []
  for token in documento:
    tokens.append(token.lemma_)

  tokens = [palavra for palavra in tokens if palavra not in stopwords and palavra not in string.punctuation]
  texto_formatado = ' '.join([str(elemento) for elemento in tokens if not elemento.isdigit()])

  return texto_formatado

In [32]:
def sumarizar_lematizacao(texto, quantidade_sentencas):
  sentencas_originais = [sentenca for sentenca in nltk.sent_tokenize(texto)]
  sentencas_formatadas = [preprocessamento_lematizacao(sentenca_original) for sentenca_original in sentencas_originais]
  matriz_similaridade = calcula_matriz_similaridade(sentencas_formatadas)
  grafo_similaridade = nx.from_numpy_array(matriz_similaridade)
  notas = nx.pagerank(grafo_similaridade)
  notas_ordenadas = sorted(((notas[i], nota) for i, nota in enumerate(sentencas_originais)), reverse=True)
  melhores_sentencas = []
  for i in range(quantidade_sentencas):
    melhores_sentencas.append(notas_ordenadas[i][1])

  return sentencas_originais, melhores_sentencas, notas_ordenadas

In [33]:
artigo.cleaned_text

'Nas eleições presidenciais americanas de 2016, a maioria das predições apontavam para a vitória de Hillary Clinton. Entretanto, a história nos mostrou o resultado oposto, e Donald Trump foi o presidente nos últimos 4 anos. Desta vez, os estatísticos reexaminaram seus modelos, para aumentar o grau de confiabilidade nos seus resultados. Nesta tentativa de otimização das predições, a inteligência artificial certamente não ficou de fora.\n\nO modelo desenvolvido pelo Dr. Hernan Makse, físico estatístico da Universidade da Cidade de Nova York, baseou suas predições em uma rede neural treinada para processar os sentimentos expressos nas redes sociais. O algoritmo fez a análise de cerca de 1 bilhão de tweets para chegar a uma estimativa dos resultados do pleito. No dia da eleição, 3 de novembro, o modelo estava indicando a vitória de Joe Biden.\n\nO Dr. Makse disse que seu trabalho começou já na eleição de 2016, e foi testado novamente nas eleições na Argentina ano passado. Desta vez, o mode

In [34]:
sentencas_originais, melhores_sentencas, _ = sumarizar(artigo.cleaned_text, 5)
visualiza_resumo(artigo.title, sentencas_originais, melhores_sentencas)

In [35]:
sentencas_originais, melhores_sentencas, _ = sumarizar_lematizacao(artigo.cleaned_text, 5)
visualiza_resumo(artigo.title, sentencas_originais, melhores_sentencas)